In [21]:
from google.colab import userdata
api_key_ali=userdata.get('Ali_OPENAI')

In [22]:
import os

# Create a folder named 'datasets' if it doesn't exist
if not os.path.exists('datasets'):
    os.makedirs('datasets')

In [100]:
import os
import re
import subprocess
import pandas as pd
from openai import OpenAI
from typing import Optional, Any

class ChatBot:
    def __init__(self, api_key: str, model: str = "gpt-4o"):
        """
        Initializes the ChatBot with OpenAI API key and model name.
        """
        self.client = OpenAI(api_key=api_key)
        self.model = model
        self.history = []

    def generate_response(self, user_message: str) -> str:
        """
        Generates a response from the chatbot based on the user's message.
        """
        self.history.append({
            "role": "user",
            "content": user_message
        })

        response = self.client.chat.completions.create(
            model=self.model,
            messages=self.history
        )

        ai_response = response.choices[0].message.content
        self.history.append({
            "role": "assistant",
            "content": ai_response
        })

        return ai_response

    def get_history(self) -> list:
        """
        Returns the conversation history.
        """
        return self.history

    def isolate_and_save_python_code(self, input_string: str, file_path: str) -> Optional[str]:
        """
        Extracts Python code block from the input string and saves it to a file.
        """
        pattern = r'```python\s*(.*?)\s*```'
        match = re.search(pattern, input_string, re.DOTALL)

        if match:
            python_code = match.group(1)
            with open(file_path, 'w') as file:
                file.write(python_code)
            return file_path
        else:
            return None

    def execute_python_script(self, script_path: str) -> Any:
        """
        Executes a local Python script and captures its output.
        """
        try:
            result = subprocess.run(['python', script_path], capture_output=True, text=True, check=True)
            return result.stdout
        except subprocess.CalledProcessError as e:
            print(f"🔴 Error executing script:\n{e.stderr}")
            return None

    def replace_synthetic_with_csv_and_columns(self, code_string: str, csv_filename: str) -> str:
      """
      Replaces synthetic code and placeholder errors in a code block using real CSV data and user-specified columns.
      """
      import pandas as pd

      print(f"🛠️ [DEBUG] Using CSV file: {csv_filename}")

      # Read the dataset temporarily to extract column names
      df = pd.read_csv(csv_filename)
      columns = df.columns.tolist()

      print("\n📑 Columns available in your dataset:")
      for idx, col in enumerate(columns, start=1):
        print(f"{idx}. {col}")

      # Ask for target column
      target_column = input("\n🎯 Enter the name of the target column (the value you want to predict): ").strip()
      while target_column not in columns:
        print("❗ Column not found. Please enter again.")
        target_column = input("🎯 Enter the name of the target column: ").strip()

      # Ask for feature columns
      feature_input = input("\n🛠️  Enter the names of feature columns separated by commas (e.g., size,bedrooms,age): ").strip()
      features = [col.strip() for col in feature_input.split(",")]

      for feature in features:
        if feature not in columns:
            raise ValueError(f"❗ Feature '{feature}' not found in dataset columns!")

      # Prepare Python-compatible list of features
      features_list = "[" + ", ".join([f"'{col}'" for col in features]) + "]"

      # Injected replacement code
      csv_injection = (
       "import pandas as pd\n"
       "from sklearn.model_selection import train_test_split\n"
       "from sklearn.linear_model import LinearRegression\n"
       "from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score\n"
       "import matplotlib.pyplot as plt\n\n"
       "# Load dataset\n"
       f"df = pd.read_csv(r'{csv_filename}')\n"
       "df.columns = df.columns.str.strip()\n\n"
       "# Select features and target\n"
       f"X = df[{features_list}]\n"
       f"y = df['{target_column}']\n\n"
       "# Split into train/test sets\n"
       "X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)\n\n"
       "# Train model\n"
       "model = LinearRegression()\n"
       "model.fit(X_train, y_train)\n\n"
       "# Make predictions\n"
       "y_pred = model.predict(X_test)\n\n"
       "# Evaluate model\n"
       "mse = mean_squared_error(y_test, y_pred)\n"
       "mae = mean_absolute_error(y_test, y_pred)\n"
       "r2 = r2_score(y_test, y_pred)\n\n"
       # Print evaluation metrics in readable format\n"
       "print('\\n📊 Model Evaluation Summary:')\n"
       "print(f'- Mean Squared Error (MSE): {mse:.2f}')\n"
       "print(f'- Mean Absolute Error (MAE): {mae:.2f}')\n"
       "print(f'- R-squared (R²) Score: {r2:.2f}')\n"
      )




      # Replace synthetic data block if present
      pattern = r"# Example only: generate synthetic data.*?# Feature selection"
      new_code = re.sub(pattern, csv_injection, code_string, flags=re.DOTALL)

      # 🔧 Force replace common broken patterns from bot output
      # 💥 REPLACE common wrong lines from bot
      new_code = new_code.replace("df.drop('target', axis=1)", f"df[{features_list}]")
      new_code = new_code.replace("data = pd.read_csv(input())", f"df = pd.read_csv(r'{csv_filename}')")
      new_code = new_code.replace("X = data[['feature1', 'feature2']]", f"X = df[{features_list}]")
      new_code = new_code.replace("y = data['target']", f"y = df['{target_column}']")
      new_code = new_code.replace("X = df.drop('target', axis=1)", f"X = df[{features_list}]")
      new_code = new_code.replace("y = df['target']", f"y = df['{target_column}']")
      new_code = new_code.replace("def load_dataset():", "# removed load_dataset")
      new_code = re.sub(r"def load_dataset\(\):.*?return pd\.DataFrame\(data\)", "", new_code, flags=re.DOTALL)
      new_code = new_code.replace("df = load_dataset()", f"df = pd.read_csv(r'{csv_filename}')")


      return new_code




    def run_openai_chatbot(self):
        """
        Main loop to run the OpenAI-powered chatbot interaction.
        """
        prompt = input("🧑 Human: ")

        while 'EXIT' not in prompt:
            response = self.generate_response(user_message=prompt)

            print("🤖 Bot: ", response)

            if '```python' in response:
                file_name = input("💻 What name do you want to save for this script? ")
                file_name = "_".join(file_name.split())

                # Ask if user wants real CSV
                use_csv = input("📂 Do you want to use a CSV file instead of synthetic data? (Y/N): ")

                if use_csv.upper() == 'Y':
                    dataset_folder = "datasets"
                    if not os.path.exists(dataset_folder):
                        os.makedirs(dataset_folder)

                    available_csvs = [f for f in os.listdir(dataset_folder) if f.endswith('.csv')]

                    if not available_csvs:
                        print("🚫 No CSV files found in 'datasets/'. Please upload a CSV file.")
                        return

                    print("\n📂 Available CSV files:")
                    for idx, file in enumerate(available_csvs, start=1):
                        print(f"{idx}. {file}")

                    choice = int(input("\n🛒 Enter the number corresponding to the CSV you want to use: "))

                    while choice < 1 or choice > len(available_csvs):
                        print("❗ Invalid choice. Try again.")
                        choice = int(input("🛒 Enter the number corresponding to the CSV you want to use: "))

                    csv_filename = os.path.join(dataset_folder, available_csvs[choice-1])

                    print(f"✅ You selected: {csv_filename}")

                    # Inject CSV-based code
                    modified_response = self.replace_synthetic_with_csv_and_columns(response, csv_filename)
                    self.isolate_and_save_python_code(
                        input_string=modified_response,
                        file_path=f"{file_name}.py"
                    )

                else:
                    # Save original code if no CSV used
                    self.isolate_and_save_python_code(
                        input_string=response,
                        file_path=f"{file_name}.py"
                    )

                # Ask to execute the script
                execution = input("💻 Do you want to execute this script? Enter 'Y' or 'N': ")
                if execution.upper() == 'Y':
                    output = self.execute_python_script(f"{file_name}.py")
                    if output is not None:
                        print("🤖 Bot Output: ")
                        print(output)
                    else:
                        print("Failed to execute the script.")

            print("NOTE: Enter 'EXIT' if you want to quit the program.")
            print()
            print()
            print()

            prompt = input("🧑 Human: ")


In [105]:
if __name__ == "__main__":
    api_key = api_key_ali
    bot = ChatBot(api_key=api_key, model="gpt-4o")
    bot.run_openai_chatbot()

🧑 Human: Use the file datasets/Housing.csv and build a linear regression model to predict price using the features area, bedrooms, and bathrooms. Just show me the evaluation metrics after execution
🤖 Bot:  I'm unable to execute code directly or access external files like `datasets/Housing.csv`. However, I can guide you through the process of building a linear regression model using Python, assuming you have the file on your local machine. Here's how you can do it using pandas, scikit-learn, and numpy for evaluation:

```python
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# Load the dataset
file_path = 'datasets/Housing.csv'
df = pd.read_csv(file_path)

# Select features and target variable
X = df[['area', 'bedrooms', 'bathrooms']]
y = df['price']

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, 